In [14]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)

2.3.0


In [4]:
import numpy as np;
import pandas as pd;

In [5]:
multi_data = pd.read_csv('../data/zri_multifamily_v2.csv', index_col = 0);

In [19]:
ZONE = 1013;
SPLIT = 60; # 2014-2018: training, 2019: testing.
BATCH_SIZE = 8;
WINDOW_SIZE = 12;

In [11]:
single_city_data = multi_data[multi_data.index == ZONE];

In [12]:
single_city_series = np.array(single_city_data["zri"])

In [17]:
single_city_train = single_city_series[:SPLIT];
single_city_test = single_city_series[SPLIT:];

In [18]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
  dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
  dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
  dataset = dataset.batch(batch_size).prefetch(1)
  return dataset

In [21]:
temp = windowed_dataset(single_city_train, WINDOW_SIZE, BATCH_SIZE, 4)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [22]:
temp

<PrefetchDataset shapes: ((None, None), (None,)), types: (tf.float64, tf.float64)>